In [ ]:
/* Plan:
1. Wyjątek.
2. Składnia.
3. Przechwytywanie.
4. Ślad stosu.
5. Rzucanie.
6. Dziedziczenie.
7. Tworzenie.
8. Popularne.
9. Trudne.
*/

In [ ]:
/* Ad 1. Wyjątek.
-> problem napotkany przy wykonaniu programu
-> umożliwia zgrabną reakcję na problem
-> reakcja na nieoczekiwaną sytuację
-> wyjątki są oczekiwane podczas wykonania programu


"they are the bread and butter of modern programming languages" ~Jeff Atwood

obsługa wyjątków dla laików (60 min, polecam tempo 2x): https://www.youtube.com/watch?v=LSkbnpjCEkk
*/

In [ ]:
/* Ad 2. Składnia. */

try {
    // Kod rzucający wyjątek
}
catch (Exception e) {
    // Obsługa wyjątku
}
finally {
    // Kod, który zostanie wykonany ZAWSZE
}

In [ ]:
/* Ad 3. Przechwytywanie. */

In [ ]:
static void ParentMethod(System.IO.StreamReader reader) {
    Method(reader);
}

// Poniższa funkcja rzuca wyjątek typu: ArgumentOutOfRangeException
static void Method(System.IO.StreamReader reader) {
    var line = reader.ReadLine();
    var lastChar = line.ElementAt(line.Length); // Próbujemy odczytać literę spoza zakresu!!!
    Console.WriteLine("Last character is: {0}", lastChar);
}

In [ ]:
// Przykład P1: Klasycznie.
static void GrandparentMethod() {
    System.IO.StreamReader reader = null;

    try {
        reader = new System.IO.StreamReader("adventofcodeiscoming.txt");
        ParentMethod(reader);
        Console.WriteLine("This cannot happen.");
    }
    catch (Exception e) {
        Console.WriteLine("Exception occured: {0}", e.Message);
    }
    finally {
        reader?.Close();
        Console.WriteLine("Anyway, this is also executed.");
    }
}

GrandparentMethod();

Exception occured: Specified argument was out of the range of valid values. (Parameter 'index')
Anyway, this is also executed.


In [ ]:
// Przykład P2: Wielokrotny catch.
static void GrandparentMethod() {
    System.IO.StreamReader reader = null;

    try {
        reader = new System.IO.StreamReader("adventofcodeiscoming.txt");
        ParentMethod(reader);
    }
    catch (IndexOutOfRangeException e) {
        Console.WriteLine("IndexOutOfRangeException occured: {0}", e.Message);
    }
    catch (ArgumentOutOfRangeException e) {
        Console.WriteLine("ArgumentOutOfRangeException occured: {0}", e.Message);
    }
    catch (Exception e) {
        Console.WriteLine("Exception occured: {0}", e.Message);
    }
    finally {
        reader?.Close();
        Console.WriteLine("Anyway, this is also executed.");
    }
}

GrandparentMethod();

ArgumentOutOfRangeException occured: Specified argument was out of the range of valid values. (Parameter 'index')
Anyway, this is also executed.


In [ ]:
// Przykład P3: Wielokrotny catch i filtry.
static void GrandparentMethod() {
    var checkForArgumentOutOfRange = false;
    var anotherCheckForArgumentOutOfRange = true;
    var checkForException = true;
    System.IO.StreamReader reader = null;

    try {
        reader = new System.IO.StreamReader("adventofcodeiscoming.txt");
        ParentMethod(reader);
    }
    catch (IndexOutOfRangeException e) {
        Console.WriteLine("IndexOutOfRangeException occured: {0}", e.Message);
    }
    catch (ArgumentOutOfRangeException e) when (checkForArgumentOutOfRange) {
        Console.WriteLine("First ArgumentOutOfRangeException occured: {0}", e.Message);
    }
    catch (ArgumentOutOfRangeException e) when (anotherCheckForArgumentOutOfRange) {
        Console.WriteLine("Second ArgumentOutOfRangeException occured: {0}", e.Message);
    }
    catch (Exception e) when (checkForException) {
        Console.WriteLine("Exception occured: {0}", e.Message);
    }
    finally {
        reader?.Close();
        Console.WriteLine("Anyway, this is also executed.");
    }
}

GrandparentMethod();

Second ArgumentOutOfRangeException occured: Specified argument was out of the range of valid values. (Parameter 'index')
Anyway, this is also executed.


In [ ]:
// Przykład P4: Życiowy.
using System.Net;

WebClient wc = null;
try
{
    wc = new WebClient();
    var resultData = wc.DownloadString("https://github.com/gryszardd");
    Console.WriteLine("OK");
}
catch (WebException ex) when (ex.Status == WebExceptionStatus.ConnectFailure)
{
    Console.WriteLine("ConnectFailure occured");
    //code specifically for a WebException ConnectFailure
}
catch (WebException ex) when ((ex.Response as HttpWebResponse)?.StatusCode == HttpStatusCode.NotFound)
{
    Console.WriteLine("HttpNotFound occured");
    //code specifically for a WebException NotFound
}
catch (WebException ex) when ((ex.Response as HttpWebResponse)?.StatusCode == HttpStatusCode.InternalServerError)
{
    Console.WriteLine("InternalServerError occured");
    //code specifically for a WebException InternalServerError
}
finally
{
    wc?.Dispose();
}

HttpNotFound occured


In [ ]:
/* Ad 4. Ślad stosu.

   pierwsza linia:  metoda, w której wyrzucono wyjątek
   druga linia:     metoda, która wywołała powyższą metodę
   trzecia linia:   ...
   ...
   ostatnia linia:  metoda, która obsłużyła wyjątek
*/

Exception e;
string output;

output = e.Message;    // tresć wyjątku
output = e.StackTrace; // ślad stosu
output = e.ToString(); // klasa, treść i ślad stosu

In [ ]:
// Przykład P5: To samo.
static void GrandparentMethod() {
    System.IO.StreamReader reader = null;
    
    try {
        reader = new System.IO.StreamReader("adventofcodeiscoming.txt");
        ParentMethod(reader);
        Console.WriteLine("This cannot happen.");
    }
    catch (Exception e) {
        Console.WriteLine("Exception occured: {0}", e.ToString());
    }
    finally {
        reader?.Close();
        Console.WriteLine("Anyway, this is also executed.");
    }
}

GrandparentMethod();

Exception occured: System.ArgumentOutOfRangeException: Specified argument was out of the range of valid values. (Parameter 'index')
   at System.Linq.ThrowHelper.ThrowArgumentOutOfRangeException(ExceptionArgument argument)
   at System.Linq.Enumerable.ElementAt[TSource](IEnumerable`1 source, Int32 index)
   at Submission#6.Method(StreamReader reader)
   at Submission#6.ParentMethod(StreamReader reader)
   at Submission#12.GrandparentMethod()
Anyway, this is also executed.


In [ ]:
/* Przykład P6: Aplikacja konsolowa.
Nieobsługiwane wyjątki trafiają do funkcji Main i powodują, że program się wysypuje.

Unhandled exception. System.ApplicationException: Randomowy wyjątek.
   at ConsoleApp1.ExceptionMaker.ThrowException() in C:\Users\rys\source\repos\ConsoleApp1\ExceptionMaker.cs:line 13
   at ConsoleApp1.Program.Main(String[] args) in C:\Users\rys\source\repos\ConsoleApp1\Program.cs:line 9

*/

In [ ]:
/* Przykład P7: Aplikacja WinForms.

System.FormatException: Nieprawidłowy format ciągu wejściowego.
   w System.Number.StringToNumber(String str, NumberStyles options, NumberBuffer& number, NumberFormatInfo info, Boolean parseDecimal)
   w System.Number.ParseInt32(String s, NumberStyles style, NumberFormatInfo info)
   w System.Int32.Parse(String s)
   w WindowsFormsApp1.DateConverter.buttonSetDateInCalendar_Click(Object sender, EventArgs e) w C:\Users\rys\source\repos\WindowsFormsApp1\DateConverter.cs:wiersz 23
   w System.Windows.Forms.Control.OnClick(EventArgs e)
   w System.Windows.Forms.Button.OnClick(EventArgs e)
   w System.Windows.Forms.Button.PerformClick()
   w System.Windows.Forms.Form.ProcessDialogKey(Keys keyData)
   w System.Windows.Forms.Control.ProcessDialogKey(Keys keyData)
   w System.Windows.Forms.Control.PreProcessMessage(Message& msg)
   w System.Windows.Forms.Control.PreProcessControlMessageInternal(Control target, Message& msg)
   w System.Windows.Forms.Application.ThreadContext.PreTranslateMessage(MSG& msg)

*/

In [ ]:
/* Przykład P8: Aplikacja webowa.

[InvalidOperationException: A specified Include path is not valid. The EntityType 'WebApplication2.Models.Animal' does not declare a navigation property with the name 'NotExistingTable'.]
   System.Data.Entity.Core.Objects.Internal.ObjectFullSpanRewriter.ConvertSpanPath(SpanPathInfo parentInfo, List`1 navPropNames, Int32 pos) +398
   System.Data.Entity.Core.Objects.Internal.ObjectFullSpanRewriter..ctor(DbCommandTree tree, DbExpression toRewrite, Span span, AliasGenerator aliasGenerator) +293
   System.Data.Entity.Core.Objects.Internal.ObjectSpanRewriter.TryRewrite(DbQueryCommandTree tree, Span span, MergeOption mergeOption, AliasGenerator aliasGenerator, DbExpression& newQuery, SpanIndex& spanInfo) +81
   System.Data.Entity.Core.Objects.Internal.ObjectQueryExecutionPlanFactory.Prepare(ObjectContext context, DbQueryCommandTree tree, Type elementType, MergeOption mergeOption, Boolean streaming, Span span, IEnumerable`1 compiledQueryParameters, AliasGenerator aliasGenerator) +71
   System.Data.Entity.Core.Objects.EntitySqlQueryState.GetExecutionPlan(Nullable`1 forMergeOption) +486
   System.Data.Entity.Core.Objects.<>c__DisplayClass7.<GetResults>b__6() +39
   System.Data.Entity.Core.Objects.ObjectContext.ExecuteInTransaction(Func`1 func, IDbExecutionStrategy executionStrategy, Boolean startLocalTransaction, Boolean releaseConnectionOnSuccess) +288
   System.Data.Entity.Core.Objects.<>c__DisplayClass7.<GetResults>b__5() +154
   System.Data.Entity.SqlServer.DefaultSqlExecutionStrategy.Execute(Func`1 operation) +189
   System.Data.Entity.Core.Objects.ObjectQuery`1.GetResults(Nullable`1 forMergeOption) +279
   System.Data.Entity.Core.Objects.ObjectQuery`1.<System.Collections.Generic.IEnumerable<T>.GetEnumerator>b__0() +11
   System.Data.Entity.Internal.LazyEnumerator`1.MoveNext() +45
   System.Collections.Generic.List`1..ctor(IEnumerable`1 collection) +186
   System.Linq.Enumerable.ToList(IEnumerable`1 source) +61
   WebApplication2.Controllers.HomeController.Index() in C:\Users\rys\source\repos\WebApplication2\Controllers\HomeController.cs:26
   lambda_method(Closure , ControllerBase , Object[] ) +62
   System.Web.Mvc.ActionMethodDispatcher.Execute(ControllerBase controller, Object[] parameters) +14
   System.Web.Mvc.ReflectedActionDescriptor.Execute(ControllerContext controllerContext, IDictionary`2 parameters) +157
   System.Web.Mvc.ControllerActionInvoker.InvokeActionMethod(ControllerContext controllerContext, ActionDescriptor actionDescriptor, IDictionary`2 parameters) +27
   System.Web.Mvc.Async.<>c.<BeginInvokeSynchronousActionMethod>b__9_0(IAsyncResult asyncResult, ActionInvocation innerInvokeState) +22
   System.Web.Mvc.Async.WrappedAsyncResult`2.CallEndDelegate(IAsyncResult asyncResult) +29
   System.Web.Mvc.Async.WrappedAsyncResultBase`1.End() +49
   System.Web.Mvc.Async.AsyncControllerActionInvoker.EndInvokeActionMethod(IAsyncResult asyncResult) +32
   System.Web.Mvc.Async.<>c__DisplayClass11_0.<InvokeActionMethodFilterAsynchronouslyRecursive>b__0() +58
   System.Web.Mvc.Async.<>c__DisplayClass11_2.<InvokeActionMethodFilterAsynchronouslyRecursive>b__2() +228
   System.Web.Mvc.Async.<>c__DisplayClass7_0.<BeginInvokeActionMethodWithFilters>b__1(IAsyncResult asyncResult) +10
   System.Web.Mvc.Async.WrappedAsyncResult`1.CallEndDelegate(IAsyncResult asyncResult) +10
   System.Web.Mvc.Async.WrappedAsyncResultBase`1.End() +49
   System.Web.Mvc.Async.AsyncControllerActionInvoker.EndInvokeActionMethodWithFilters(IAsyncResult asyncResult) +34
   System.Web.Mvc.Async.<>c__DisplayClass3_6.<BeginInvokeAction>b__4() +35
   System.Web.Mvc.Async.<>c__DisplayClass3_1.<BeginInvokeAction>b__1(IAsyncResult asyncResult) +100
   System.Web.Mvc.Async.WrappedAsyncResult`1.CallEndDelegate(IAsyncResult asyncResult) +10
   System.Web.Mvc.Async.WrappedAsyncResultBase`1.End() +49
   System.Web.Mvc.Async.AsyncControllerActionInvoker.EndInvokeAction(IAsyncResult asyncResult) +27
   System.Web.Mvc.<>c.<BeginExecuteCore>b__152_1(IAsyncResult asyncResult, ExecuteCoreState innerState) +11
   System.Web.Mvc.Async.WrappedAsyncVoid`1.CallEndDelegate(IAsyncResult asyncResult) +29
   System.Web.Mvc.Async.WrappedAsyncResultBase`1.End() +49
   System.Web.Mvc.Controller.EndExecuteCore(IAsyncResult asyncResult) +45
   System.Web.Mvc.<>c.<BeginExecute>b__151_2(IAsyncResult asyncResult, Controller controller) +13
   System.Web.Mvc.Async.WrappedAsyncVoid`1.CallEndDelegate(IAsyncResult asyncResult) +22
   System.Web.Mvc.Async.WrappedAsyncResultBase`1.End() +49
   System.Web.Mvc.Controller.EndExecute(IAsyncResult asyncResult) +26
   System.Web.Mvc.Controller.System.Web.Mvc.Async.IAsyncController.EndExecute(IAsyncResult asyncResult) +10
   System.Web.Mvc.<>c.<BeginProcessRequest>b__20_1(IAsyncResult asyncResult, ProcessRequestState innerState) +28
   System.Web.Mvc.Async.WrappedAsyncVoid`1.CallEndDelegate(IAsyncResult asyncResult) +29
   System.Web.Mvc.Async.WrappedAsyncResultBase`1.End() +49
   System.Web.Mvc.MvcHandler.EndProcessRequest(IAsyncResult asyncResult) +28
   System.Web.Mvc.MvcHandler.System.Web.IHttpAsyncHandler.EndProcessRequest(IAsyncResult result) +9
   System.Web.CallHandlerExecutionStep.System.Web.HttpApplication.IExecutionStep.Execute() +9850009
   System.Web.HttpApplication.ExecuteStepImpl(IExecutionStep step) +50
   System.Web.HttpApplication.ExecuteStep(IExecutionStep step, Boolean& completedSynchronously) +163

*/

In [ ]:
// Ad 5. Rzucanie.

throw new Exception("Random exception.");

In [ ]:
// Przykład P9: Samolot.
class Airplane {
    public void ChangeHeight(int meters) {
        if (currentHeight + meters > maxHeight) {
            throw new Exception("Sorry, this airplane cannot fly so high.");
        }
        if (currentHeight + meters < 0) {
            throw new Exception("Sorry, this airplane is not an underground.");
        }

        currentHeight += meters;
        Console.WriteLine("Airplane is now on height: {0} meters.", currentHeight);
    }

    public Airplane(int currentHeight, int maxHeight) {
        this.currentHeight = currentHeight;
        this.maxHeight = maxHeight;
    }

    private int currentHeight;
    private int maxHeight;
}

In [ ]:
void SteerThePlane() {
    var airplane = new Airplane(200, 300);
    airplane.ChangeHeight(100);
    airplane.ChangeHeight(100);
}

// Obsługa wyjątku, aplikacja dalej działa.
try {
    SteerThePlane();
}
catch (Exception e) {
    Console.WriteLine("Exception occured: {0}", e.Message);
}

Airplane is now on height: 300 meters.
Exception occured: Sorry, this airplane cannot fly so high.


In [ ]:
void SteerThePlane() {
    var airplane = new Airplane(200, 300);
    airplane.ChangeHeight(-200);
    airplane.ChangeHeight(-200);
}

// Zmiana stanu programu i przekazanie wyjątku.
// Duża szansa, że program się skraszuje.
var isBroken = false;
try {
    SteerThePlane();
}
catch (Exception e) {
    isBroken = true;
    Console.WriteLine("Exception occured: {0}", e.Message);
    throw;
    // throw e; spowodowałby, że stracimy informację o całym śladzie stosu
}

Airplane is now on height: 0 meters.
Exception occured: Sorry, this airplane is not an underground.


Error: System.Exception: Sorry, this airplane is not an underground.
   at Submission#13.Airplane.ChangeHeight(Int32 meters)
   at Submission#15.SteerThePlane()
   at Submission#15.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
// Ad 6. Dziedziczenie.
// Wybierz plik dziedziczenie.png
System.Diagnostics.Process.Start("explorer.exe", @".");

In [ ]:
/* Ad 7. Tworzenie.

Gdy pojawia się problem, rzucamy ApplicationException lub Exception,
chyba, że zamierzamy inaczej obsłużyć konkretny typ wyjątku.

Na przykład: Zapisać inną informację do logów w danej sytuacji.
Na przykład: Wysłać SMS do programisty, bo w systemie wydarzyło się coś krytycznego i trzeba przyjechać do biura.

źródło: https://stackify.com/csharp-exception-handling-best-practices/
*/

In [ ]:
/* 
Dobrą praktyka jest przechwytywanie jak najbardziej szczegółowych wyjątków.

Przykładowy problem:
Przechwytujemy Exception, robimy coś i kontynuujemy działanie aplikacji.
Pojawia się OutOfMemoryException, które też zostanie przechwycone (podklasa Exception).
Jak obsłużyć taki problem? 
Czy to nie doprowadzi do kolejnych problemów, jeśli będziemy kontynuować działanie aplikacji?

Bezpieczniej przechwycić ApplicationException, bo wtedy żaden systemowy wyjątek tu nie wpadnie.
Najlepiej używać własnych wyjątków, dla konkretnych sytuacji.


Łapcie wyjątek, tylko gdy:
1. Wiecie dokładnie jak go obsłużyć
2. Wyrzucicie go za chwilę, ale chcecie coś zrobić lokalnie (np. zmienić stan programu, zrobić logi).
3. (*) Potrzebujecie przetransportować go do innego wątku

źródło: https://www.generacodice.com/en/articolo/332961/Why-is-the-Catch(Exception)-almost-always-a-bad-Idea
*/

In [ ]:
// Przykład P11: Nowy samolot.
// Można się trochę pobawić ;)
class IncorrectAltitudeException : ApplicationException {
    public IncorrectAltitudeException(string msg) : base(msg) {}
}

class ToHighAltitudeException : IncorrectAltitudeException {
    // Jeśli chcemy rzucać wyjątek z pewną treścią, to należy zaimplementować odpowiedni konstruktor.
    public ToHighAltitudeException(string msg) : base(msg) {}
}
class ToLowAltitudeException : IncorrectAltitudeException {
    public ToLowAltitudeException(string msg) : base(msg) {}
}

class Airplane {
    public void ChangeHeight(int meters) {
        var newHeight = currentHeight + meters;

        if (newHeight > maxHeight) {
            throw new ToHighAltitudeException($"Próba podwyższenia wysokości do {newHeight}, " + 
                "a maksymalna to {maxHeight}.");
        }
        if (newHeight < 0) {
            throw new ToLowAltitudeException($"Próba obniżenia wysokości na {newHeight}, a minimalna to 0.");
        }

        currentHeight += meters;
        Console.WriteLine("Airplane is now on height: {0} meters.", currentHeight);
    }

    public Airplane(int currentHeight, int maxHeight) {
        this.currentHeight = currentHeight;
        this.maxHeight = maxHeight;
    }

    private int currentHeight;
    private int maxHeight;
}

In [ ]:
void SteerThePlane() {
    var airplane = new Airplane(200, 300);
    airplane.ChangeHeight(-200);
    airplane.ChangeHeight(-10);
}


var lackOfAirAlert = false;
var emergencyCallRequired = false;

try {
    SteerThePlane();
}
// "Wiecie dokładnie jak go obsłużyć" oraz "zamierzamy inaczej obsłużyć konkretny typ wyjątku"
catch (ToHighAltitudeException e) {
    lackOfAirAlert = true;
    Console.WriteLine("ToHighAltitudeException occured: {0}", e.Message);
}
catch (ToLowAltitudeException e) {
    emergencyCallRequired = true;
    Console.WriteLine("ToLowAltitudeException occured: {0}", e.Message);
}

Airplane is now on height: 0 meters.
ToLowAltitudeException occured: Próba obniżenia wysokości na -10, a minimalna to 0.


In [ ]:
/* Ad 8. Popularne. (https://docs.microsoft.com/en-us/dotnet/api/system.exception?view=net-6.0)

NullReferenceException      Trying to dereference a null object reference.
ArgumentException           A non-null argument that is passed to a method is invalid.
ArgumentNullException       An argument that is passed to a method is null.
ArgumentOutOfRangeException An argument is outside the range of valid values.
DirectoryNotFoundException  Part of a directory path is not valid.
DivideByZeroException       The denominator in an integer or Decimal division operation is zero.
DriveNotFoundException      A drive is unavailable or does not exist.
FileNotFoundException       A file does not exist.
FormatException             A value is not in an appropriate format to be converted from a string 
                            by a conversion method such as Parse.
IndexOutOfRangeException    An index is outside the bounds of an array or collection.
InvalidOperationException   A method call is invalid in an object's current state.
KeyNotFoundException        The specified key for accessing a member in a collection cannot be found.
NotImplementedException     A method or operation is not implemented.
ObjectDisposedException     An operation is performed on an object that has been disposed.
OverflowException           An arithmetic, casting, or conversion operation results in an overflow.
*/

In [ ]:
// NullReferenceException       Trying to dereference a null object reference.
List<int> list;
list.Add(2);

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#18.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
// ArgumentException	        A non-null argument that is passed to a method is invalid.
var reader = new System.IO.StreamReader("");

Error: System.ArgumentException: Empty path name is not legal.
   at System.IO.StreamReader.ValidateArgsAndOpenPath(String path, Encoding encoding, Int32 bufferSize)
   at System.IO.StreamReader..ctor(String path)
   at Submission#19.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
// ArgumentNullException	    An argument that is passed to a method is null.
var newString = "python is the best".Replace("python", "dotnet");
Console.WriteLine(newString);

newString = "python is the best".Replace(null, "dotnet");
Console.WriteLine(newString);

dotnet is the best


Error: System.ArgumentNullException: Value cannot be null. (Parameter 'oldValue')
   at System.String.Replace(String oldValue, String newValue)
   at Submission#20.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
// ArgumentOutOfRangeException	An argument is outside the range of valid values.
var stringlist = new List<string>(-1);

Error: System.ArgumentOutOfRangeException: Non-negative number required. (Parameter 'capacity')
   at System.Collections.Generic.List`1..ctor(Int32 capacity)
   at Submission#21.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
// DirectoryNotFoundException	Part of a directory path is not valid.
System.IO.Directory.Delete("this is not a valid path");

Error: System.IO.DirectoryNotFoundException: Could not find a part of the path 'c:\Users\rys\source\exceptions\this is not a valid path'.
   at System.IO.FileSystem.RemoveDirectoryInternal(String fullPath, Boolean topLevel, Boolean allowDirectoryNotEmpty)
   at System.IO.FileSystem.RemoveDirectory(String fullPath, Boolean recursive)
   at System.IO.Directory.Delete(String path)
   at Submission#22.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
// DivideByZeroException	    The denominator in an integer or Decimal division operation is zero.
int a = 2;
int b = 0;

var value = a / b;

Error: System.DivideByZeroException: Attempted to divide by zero.
   at Submission#23.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
// DriveNotFoundException	    A drive is unavailable or does not exist.
// code from: https://kodify.net/csharp/computer-drive/inaccessible-drives/#handle-c-exceptions-caused-by-inaccessible-drives

// public class DriveNotFoundException : System.IO.IOException
var drive = new System.IO.DriveInfo("C");

// Czy w tym miejscu użyć const int?
var bytesInAGB = 1024 * 1024 * 1024;
Console.WriteLine("Free space in {1}: {0:0.00} GB", (double)drive.AvailableFreeSpace / bytesInAGB, drive.Name[0]);

// Nie posiadam dysku X
drive = new System.IO.DriveInfo("X");
Console.WriteLine("Free space in {1}: {0:0.00} GB", (double)drive.AvailableFreeSpace / bytesInAGB, drive.Name[0]);

Free space in C: 59,15 GB


Error: System.IO.DriveNotFoundException: Could not find the drive 'X:\'. The drive might not be ready or might not be mapped.
   at System.IO.DriveInfo.get_AvailableFreeSpace()
   at Submission#24.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
// FileNotFoundException	    A file does not exist.
using System.IO;

File.Open("this is not a valid path", FileMode.Open);

Error: System.IO.FileNotFoundException: Could not find file 'c:\Users\rys\source\exceptions\this is not a valid path'.
File name: 'c:\Users\rys\source\exceptions\this is not a valid path'
   at System.IO.FileStream.ValidateFileHandle(SafeFileHandle fileHandle)
   at System.IO.FileStream.CreateFileOpenHandle(FileMode mode, FileShare share, FileOptions options)
   at System.IO.FileStream..ctor(String path, FileMode mode, FileAccess access, FileShare share, Int32 bufferSize, FileOptions options)
   at System.IO.FileStream..ctor(String path, FileMode mode, FileAccess access, FileShare share)
   at System.IO.File.Open(String path, FileMode mode)
   at Submission#25.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToke

In [ ]:
// FormatException			    A value is not in an appropriate format to be converted from a string 
//                              by a conversion method such as Parse.
var value = int.Parse("trzy");

Error: System.FormatException: Input string was not in a correct format.
   at System.Number.ThrowOverflowOrFormatException(ParsingStatus status, TypeCode type)
   at System.Number.ParseInt32(ReadOnlySpan`1 value, NumberStyles styles, NumberFormatInfo info)
   at System.Int32.Parse(String s)
   at Submission#26.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
// IndexOutOfRangeException		An index is outside the bounds of an array or collection.
var values = new int[] { 2, 1, 3, 7 };
Console.WriteLine(values[4]);

Error: System.IndexOutOfRangeException: Index was outside the bounds of the array.
   at Submission#27.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
// InvalidOperationException	A method call is invalid in an object's current state.
var numbers = new List<int>() { 1, 2, 3, 4, 5 };
foreach (var number in numbers) {
    int square = (int) Math.Pow(number, 2);
    Console.WriteLine("Adding {0} to the collection...\n", square);
    numbers.Add(square);
}

Adding 1 to the collection...



Error: System.InvalidOperationException: Collection was modified; enumeration operation may not execute.
   at System.Collections.Generic.List`1.Enumerator.MoveNextRare()
   at Submission#28.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
// KeyNotFoundException	    	The specified key for accessing a member in a collection cannot be found.

// public class KeyNotFoundException : SystemException
var colorCodes = new Dictionary<string, ConsoleColor> { 
    { "@R", ConsoleColor.Blue },
    { "@G", ConsoleColor.Red },
    { "@B", ConsoleColor.Green }
};

Console.BackgroundColor = colorCodes["@Y"];

Error: System.Collections.Generic.KeyNotFoundException: The given key '@Y' was not present in the dictionary.
   at System.Collections.Generic.Dictionary`2.get_Item(TKey key)
   at Submission#29.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
// NotImplementedException	  	A method or operation is not implemented.

// Specjalnie dałem zwracany typ int, jeśli rzucamy wyjątek, to nie musimy zwracać wyniku.
static int Ficzer()
{
    // Not developed yet.
    throw new NotImplementedException();
}

Ficzer();

Error: System.NotImplementedException: The method or operation is not implemented.
   at Submission#30.Ficzer()
   at Submission#30.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
// ObjectDisposedException	  An operation is performed on an object that has been disposed.
System.IO.StreamReader reader;

using (reader = new System.IO.StreamReader("test.txt")) {
    var readLine = string.Empty;

    while ((readLine = reader.ReadLine()) != null) {
        Console.WriteLine(readLine);
    }
}

reader.ReadLine();

test
1
2
3


Error: System.ObjectDisposedException: Cannot read from a closed TextReader.
Object name: 'StreamReader'.
   at System.IO.StreamReader.<ThrowIfDisposed>g__ThrowObjectDisposedException|68_0()
   at System.IO.StreamReader.ReadLine()
   at Submission#31.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
// OverflowException             An arithmetic, casting, or conversion operation results in an overflow.

/*
The checked keyword is used to explicitly enable overflow checking 
for integral-type arithmetic operations and conversions.
*/
checked {
    int value = System.Int32.MaxValue;
    int square = value * value;
    Console.WriteLine("{0} ^ 2 = {1}", value, square);
}

Error: System.OverflowException: Arithmetic operation resulted in an overflow.
   at Submission#32.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
// Ad 9. Trudne.
// Patrz: projekt ConsoleApplication1